In [1]:
%load_ext autoreload
%autoreload 2

When we realised we had to get a new API to find local concerts before doing any model work, I did some research and found this Zyla Lab API. Here, I was figuring out how it worked

In [38]:
# not in use

import requests

city = 'Norwich'
min_date = '2023-03-01'
max_date = '2023-03-31'

url = 'https://zylalabs.com/api/94/music+gigs+and+concerts+tracker+api/149/get+upcoming+concerts+by+location'

headers = {
    'Authorization': 'Bearer 1003|T1pt4y6iLx3ROZLT8g91Oo3U0tS05jthjq5sLx2m',
    'Accept-Language': 'en-US'
}

params = {'name' : city, 'minDate' : min_date, 'maxDate' : max_date, 'page' : 1}

response = requests.request("GET", url, headers=headers, params=params).json()
    


artist_list = []

for i in response['data']:
    if i['@type'] == 'MusicEvent':
        artist_list.append(i['name'])
        
artist_list
'Pile' in artist_list

False

The above request worked, however I was unhappy with the fact it couldnt really be used for every page of the request, limiting the output to 50 artists. I could have made a loop that iterated through the pages inside the function params, but I thought that would be ugly, so I made 3 functions: one thats fetches a page, one that parses the JSON from the API request, and one that iterates through all the pages that contain data (the while loop stops when it finds and empty page)

In [40]:
import requests

def fetch_page(page, city, min_date, max_date):
    """
    Fetch a page of gigs from the zylab API
    """
    url = 'https://zylalabs.com/api/94/music+gigs+and+concerts+tracker+api/149/get+upcoming+concerts+by+location'

    headers = {
        'Authorization': 'Bearer 1003|T1pt4y6iLx3ROZLT8g91Oo3U0tS05jthjq5sLx2m'
}
    params = {'name' : city, 'minDate' : min_date, 'maxDate' : max_date, 'page' : page}
    response = requests.request("GET", url, headers=headers, params=params).json()
    page_dict = response['data']
    return page_dict

def parse_data(page_dict):
    """
    Parse the data from the zylab API
    """
    artist_list = []
    for i in page_dict:
        if i['@type'] == 'MusicEvent':
            artist_list.append(i['name'])
    return artist_list

In [39]:
def gather_gigs(city, min_date, max_date):
    """
    Gather all the gigs from the zylab API
    """
    page = 1
    artist_list = []
    gig_dict_list = []
    while True:
        page_dict = fetch_page(page, city, min_date, max_date)
        if len(page_dict) == 0:
            break
        artist_list.extend(parse_data(page_dict))
        gig_dict_list.extend(page_dict)
        page += 1
    return artist_list, gig_dict_list

In [41]:
artist_list, gd = gather_gigs('Norwich', '2023-03-01', '2023-03-31')

In [42]:
artist_list

['Tommy Blaize',
 'Ella Clayton',
 'Blaze Bayley',
 'Coops',
 'Coops (UK)',
 'Hannah Grae',
 'Erotic Secrets of Pompeii',
 'Charming Liars',
 'Daughtry',
 'Nathan Fake',
 'Static Dress',
 'Showmaddymaddy',
 'John Ward Trio',
 'The Oozes',
 'Badly Drawn Boy',
 'Stiff Little Fingers',
 'The Steve Hillage Band',
 'Sir DJ Corey',
 'Hayseed Dixie',
 'The Undercover Hippy',
 'The JB Conspiracy',
 'Jarrod Dickenson',
 'Black Honey',
 'Absolute Bowie',
 'Whitney - Queen of the Night',
 '2 Rooms',
 'PleasureInc.',
 'The Machine Rages On',
 'The Dolly Parton Experience',
 'Dune Rats',
 'London Concertante',
 'Len Phillips Swing Orchestra',
 'Piglet',
 'Phoebe Troup',
 'Geno Washington',
 'Down for the Count Swing Orchestra',
 'Chris Wood',
 'SKYHIGH (UK)',
 'Zion Train',
 'The love asylum']

In [54]:
'Rosie Frater-Taylor' in list

False

Here I tried finding a regex function that removed a lot of unnecessary info from the 'description' field

In [30]:
import pandas as pd
import re 
gigs_df = pd.DataFrame(gd)


regex_pattern = r"^(.*?) at [0-9-]+[T\s][0-9:\.]+"
suggested_gig = gigs_df[gigs_df['name'] == 'Pixies'.strip("'")]
suggested_gig[['description']].applymap(lambda x: re.findall(regex_pattern, x)[0] if re.findall(regex_pattern, x) else x).iloc[0][0]


'Pixies at Campo Pequeno'

In [19]:
artist_list = []
for i in gd:
    if i['@type'] == 'MusicEvent':
        artist_list.append(i['name'])

In [20]:
len(artist_list)

55

In [1]:
import pandas as pd
data = pd.read_csv('../raw_data/new_data.csv')


In [2]:
data.shape

(1204865, 24)

In [1]:
import pandas as pd
data = pd.read_csv('../raw_data/tracks_features.csv')

In [2]:
data.shape

(1204865, 24)

In [3]:
data['artists'] = data['artists'].str.lower()

In [4]:
data[data.name.str.contains('Rolling in the Deep')] 

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date
165444,2dwqluhcj6xsU0SRgmAv1a,Rolling in the Deep,Grand Ukulele,5NimF8cRkNfPpKROHq2dhu,['Jake Shimabukuro'],['69NjH5MsRLr0CX0zSlGmN3'],2,1,False,0.545,...,0.0543,0.9210,0.803000,0.0889,0.4770,121.951,289283,4.0,2012,2012-10-02
276419,3KygsMocyR6lHzMhLPW3Ha,Rolling in the Deep,The Purring Piano,7IhMjGVOlvuvGKiiGAsvTT,['Bucktown Tiger'],['5KV4trIjekiYyBtpBP6BVi'],10,1,False,0.464,...,0.0381,0.8400,0.852000,0.0826,0.0768,110.688,225587,4.0,2011,2011-09-02
529449,5iU0EBDpm6ERkBhzc2e5P2,Rolling in the Deep,Sabrosura: Música de Guatemala para los Latinos,0sVu8W1H4C81mw0NMSIMHW,['Sabrosos del Swing'],['0etZQVWHWcmneyBHSu0pEa'],10,1,False,0.714,...,0.0291,0.0966,0.000228,0.1250,0.7240,104.968,230107,4.0,2015,2015-03-11
797442,3yzRROh60Ndhc1csMJp2xv,Rolling in the Deep,Lullaby Renditions of Adele,1QKRhduQ3uuv5sXxcQ0j0K,['Rockabye Baby!'],['5bHjVR4F2Tfq4Ha6x7K6wU'],3,1,False,0.899,...,0.1040,0.7200,0.662000,0.1100,0.6880,105.001,211333,4.0,2016,2016-04-29
798944,2N3Bp4QU3JeqqQiL2geJqV,Rolling in the Deep (vocal version),Rolling in the Deep,2Jy7wRJN7Y0msbgYnw9NrO,['The Piano Guys'],['0jW6R8CVyVohuUJVcuweDI'],1,1,False,0.589,...,0.0374,0.1870,0.019100,0.1210,0.2030,109.934,240973,4.0,2012,2012-09-11
835307,2CCYsscPzaTBcGKqreuiWw,Rolling in the Deep - Live,Live in Berlin,6jbgMMUxVjHtxDqjJWT5bL,"[""Mo' Blow"", 'Nils Landgren']","['5mYza9HAQFaxpwdrdRh4fa', '6B3ZWSop1mrJd71rwF...",6,1,False,0.518,...,0.2220,0.3610,0.000034,0.6990,0.5070,130.895,360373,4.0,2016,2016-01-29
903565,2E0hnv50W0CkefOQ0goFSj,Rolling in the Deep,Funk-ish,4k2vl2wQKROOxKHrplHmCO,"['Scary Pockets', 'Sarah Dugas']","['1e16kiJQtCTveTl7TQnkFN', '00MWO9aNCxqV0zfezW...",8,1,False,0.711,...,0.0625,0.0528,0.003170,0.6260,0.7310,172.480,249432,4.0,2018,2018-05-09
933995,1CkvWZme3pRgbzaxZnTl5X,Rolling in the Deep,21,1azUkThwd2HfUDdeNeT147,['Adele'],['4dpARuHxo51G3z768sgnrY'],1,1,False,0.729,...,0.0294,0.1310,0.000000,0.0527,0.5220,104.945,228293,4.0,2011,2011-01-19
1123805,5xPazRvyrkVootu8pM9vUG,Rolling in the Deep,Rolling In The Deep,6cw6QU9pF7FyQtSyJjzMua,['Adele'],['4dpARuHxo51G3z768sgnrY'],1,1,False,0.730,...,0.0298,0.1380,0.000000,0.0473,0.5070,104.948,228093,4.0,2010,2010-11-30


In [17]:
data.dtypes

id                   object
name                 object
album                object
album_id             object
artists              object
artist_ids           object
track_number          int64
disc_number           int64
explicit               bool
danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
duration_ms           int64
time_signature      float64
year                  int64
release_date         object
dtype: object

In [4]:
data.sample(10)

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date
535407,3ZPeqbFdzKDmOaMvaZdUeX,Sixteen,Kubic's Monk,5hYyQOtnh2C8gAVbKmhU8c,['pierrick pédron'],['10MQ21DbVINzGluMOuNVtp'],8,1,False,0.574,...,0.1250,0.1050,0.033100,0.0326,0.6180,122.847,317617,4.0,2012,2012-09-28
412457,78eIOdmQ6XgowQtIYvSyZr,Vida (A Klezumbia for Cory),Night Trip,0hjRaMnXMu8Eqpfw653C4E,['the new triumph'],['6YiIurMsFZTpVmzXog4OlJ'],2,1,False,0.500,...,0.0561,0.1540,0.804000,0.3240,0.7140,104.340,376333,4.0,2020,2020-06-12
837208,4EnyN8CIx6P7r9sP7yHcmE,This for the (feat. Quando Rondo),4Respect 4Freedom 4Loyalty 4WhatImportant,4tCZ22E9RnUz09OaDJfaoO,"['youngboy never broke again', 'quando rondo']","['7wlFDEWiM5OoIAt8RSli8b', '4IprNlQiJZUUJhDl0f...",9,1,True,0.815,...,0.1300,0.3050,0.000000,0.1130,0.5010,144.058,187199,4.0,2018,2018-09-14
903593,47y7beh3AmRUdAjPaRKBKp,3:45am,Circular Logic,6LaX8j7lF0Sqa5erW1IFdI,['phil vincent'],['53UAwA032WCqGUNCKwFKCh'],20,1,False,0.511,...,0.0321,0.0207,0.000075,0.1060,0.0707,120.466,593000,4.0,2002,2002-01-01
596322,2riQOk7Q3onoKEmn5DQSNV,"Girls, Girls Everywhere",Changing Faces,64cC2CVhEqe6NtjubgH5zR,['till kersting'],['5xcWeWshSPlGnL8GDf95yu'],9,1,False,0.605,...,0.0770,0.3250,0.000007,0.1290,0.7520,178.015,295373,4.0,2006,2006-04-07
895683,03idNd2jQ6pMu83N5G5NGR,Faded,Something to Say,7BkeaEk4fppxtBG2Aaxx9x,['richie kotzen'],['6nYo7m5cO64bANRvilwVBb'],4,1,False,0.544,...,0.0370,0.0374,0.000029,0.0883,0.6590,96.047,315640,4.0,2003,2003-02-11
82937,6JWR9bQ9WT8JZXb5BpvPpx,Rage Against The Vending Machine,Angels In The Chamber,38kSfCRxb9GoYha2ohQJ5e,['bad fathers'],['2owPfStl9V9AovJuXFTd7X'],16,1,False,0.593,...,0.1040,0.5610,0.000000,0.4160,0.8120,80.001,185680,4.0,2005,2005-01-01
466581,5xIDhzFpkSv8KG4AxDBOfd,Make My Own Way,Gypsy Soul,2dfDQE9HiPykwNC7y7ddvf,['cat myles & the possessions'],['7sPkOOyqWSn82oUyz4xE0a'],1,1,False,0.302,...,0.0690,0.1320,0.002360,0.0695,0.4150,170.217,283533,4.0,2015,2015-11-21
350632,34HP1HuP3u0FKnOH3b4pEY,Rods On Crutches,My Pain and Sadness Is More Sad and Painful Th...,6nHTKj76qq6Kwa3SNMhCAW,['mclusky'],['3xA9AB0f4BVjsSlyQc12iE'],11,1,False,0.684,...,0.0799,0.3930,0.000000,0.1200,0.6560,100.732,151307,4.0,2004,2004-02-23
432893,20MzkgT6PdND0aEUC5pk4p,Portuguese Solid Summer,Chad Valley,5mKhY2bJzUhCWbLHjXe0SU,['chad valley'],['1rzpgMqxYjRQo4UmWZimFc'],1,1,False,0.463,...,0.0335,0.3490,0.240000,0.1830,0.6390,93.955,215279,4.0,2010,2010


In [6]:
data['artists_clean'] = data['artists'].str.strip('[]').str.strip("'")
data['artists_clean'] = data['artists_clean'].str.split("', '")

In [7]:
matched_i_list = []
for artist in artist_list:
    song_i_list = data.index[data['artists_clean'].apply(lambda x: artist in x)]
    for song_i in song_i_list:
        matched_i_list.append(song_i)

In [14]:
len(matched_i_list)

359

In [8]:
matched_df = data.iloc[matched_i_list]

In [9]:
matched_df.loc[:, 'matched_artist'] = [[artist for artist in artists if artist in artist_list] 
                                        for artists in matched_df['artists_clean']]


/tmp/ipykernel_8443/1593719967.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_df['matched_artist'] = [[artist for artist in artists if artist in artist_list]


# Does it work?

In [10]:
from test_package.modules.nn_model import fit_nn
from test_package.modules.PCA import pca
from test_package.modules.preprocessing import preprocess_data,preprocess_numerical


def init_model(data,
               data_matched,
               n_components=7):
    """
    Initialize the model and return the model object
    """
    preproc_num = preprocess_numerical(data)
    preproc_matched_num = preprocess_numerical(data_matched)
    data_pca, data_matched_pca = pca(preproc_num, preproc_matched_num, n_components)
    song_knn = fit_nn(data_matched_pca, len(preproc_matched_num))

    return song_knn, data_pca

In [11]:
matched_df = matched_df.reset_index(drop=True)

In [12]:
song_knn, data_pca = init_model(data, matched_df)

: 

: 

In [18]:
nn_distance = pd.Series(song_knn.kneighbors(data_pca[0].reshape(1,-1),return_distance=True)[0][0])
old_index = pd.Series(song_knn.kneighbors(data_pca[0].reshape(1,-1),return_distance=True)[1][0])
nn_distance_df = pd.concat([nn_distance, old_index],axis=1)
nn_distance_df = nn_distance_df.set_index(1)

nn_distance_df

# 0 is just for testing, should be the id of the song

,0
1,
94,3.414570
0,3.444441
176,3.607784
328,3.724312
260,3.853398
...,...
144,23.574917
323,23.902208
146,24.478258


In [24]:
final_df = matched_df.join(nn_distance_df)
final_df = final_df.rename(columns={0:'nn_distance'})
final_df['matched_artist'] = final_df['matched_artist'].str[0]
artist_distance = pd.DataFrame(final_df.nn_distance.groupby(final_df.matched_artist).mean().sort_values())
best_matched_artist = artist_distance.index[0]
best_matched_artist

'Jorge Aragão'

In [66]:
gigs = pd.DataFrame(response['data'])
suggested_gig = gigs[gigs['name'] == best_matched_artist.strip("'")]#['description'][0]
suggested_gig_name = suggested_gig['description'][0][:-28]
suggested_gig_date = pd.to_datetime(suggested_gig.startDate)
suggested_gig_date


0   2023-03-09 21:30:00+00:00
Name: startDate, dtype: datetime64[ns, UTC]

In [2]:
from test_package.modules.preprocessing import generate_data_table

df = make_clean_artists(data)

In [1]:
df.head(10)

NameError: name 'df' is not defined

In [5]:
from test_package.modules.preprocessing import make_matched_df

matched_df = make_matched_df(data, artist_list)

/home/afonsofroes/code/harpallsingh/terrible-with-concerts/test_package/modules/preprocessing.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
matched_df

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,...,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date,artists_clean,matched_artist
864698,7E7fz3ASibEwtOFjjFwWtu,Minhas Andanças,Grupo Fundo de Quintal Ao Vivo - Gravado no Ca...,06pWCUA5zM85pDJMCsUOWp,"['Grupo Fundo De Quintal', 'Jorge Aragão']","['1WkZvxuA4zCcFF9GChK6Vr', '2cAooeeHKk2UvzUgIx...",5,1,False,0.585,...,0.000004,0.7900,0.7830,97.743,265587,4.0,2002,2002-07-19,"[Grupo Fundo De Quintal, Jorge Aragão]",[Jorge Aragão]
443831,33W4ROE8qHyGdrNdHD0nxV,Meditation,Meditation,3BVDfs5wK55PIDwufoGolo,"['Sebastian Gahler Trio', 'Sebastian Gahler', ...","['4v4SAX1Ni3vfh7xQJFsDSS', '3CfwkW9rDM1yyEuPRR...",1,1,False,0.288,...,0.911000,0.1050,0.1530,74.942,419133,4.0,2009,2009-01-30,"[Sebastian Gahler Trio, Sebastian Gahler, Nico...",[Sebastian Gahler]
443832,1W5VjZb5uV2WD7hlOrw9zZ,Bound,Meditation,3BVDfs5wK55PIDwufoGolo,"['Sebastian Gahler Trio', 'Sebastian Gahler', ...","['4v4SAX1Ni3vfh7xQJFsDSS', '3CfwkW9rDM1yyEuPRR...",2,1,False,0.389,...,0.900000,0.0762,0.2050,99.625,477213,3.0,2009,2009-01-30,"[Sebastian Gahler Trio, Sebastian Gahler, Nico...",[Sebastian Gahler]
443833,4AJJ6V3LthPhRs6hnyfUhy,Lost,Meditation,3BVDfs5wK55PIDwufoGolo,"['Sebastian Gahler Trio', 'Sebastian Gahler', ...","['4v4SAX1Ni3vfh7xQJFsDSS', '3CfwkW9rDM1yyEuPRR...",3,1,False,0.542,...,0.580000,0.0976,0.5570,113.887,319413,4.0,2009,2009-01-30,"[Sebastian Gahler Trio, Sebastian Gahler, Nico...",[Sebastian Gahler]
443834,0vC2nddsRl0K4fpKlaE4P5,Delusion,Meditation,3BVDfs5wK55PIDwufoGolo,"['Sebastian Gahler Trio', 'Sebastian Gahler', ...","['4v4SAX1Ni3vfh7xQJFsDSS', '3CfwkW9rDM1yyEuPRR...",4,1,False,0.425,...,0.884000,0.0987,0.0847,124.823,491093,4.0,2009,2009-01-30,"[Sebastian Gahler Trio, Sebastian Gahler, Nico...",[Sebastian Gahler]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382850,2tqKwzBTQx3xz4ieDTD8zb,Soul Transmission,The Christmas Present (Deluxe),1JDeu83sZU5nmRu2OTSk4F,['Robbie Williams'],['2HcwFjNelS49kFbfvMxQYw'],11,2,False,0.402,...,0.000022,0.0821,0.1190,198.049,254067,4.0,2019,2019-11-22,[Robbie Williams],[Robbie Williams]
382851,23oU1ASEElGxgiUSmWXeCg,I Believe in Father Christmas - Bonus Track,The Christmas Present (Deluxe),1JDeu83sZU5nmRu2OTSk4F,['Robbie Williams'],['2HcwFjNelS49kFbfvMxQYw'],12,2,False,0.646,...,0.000004,0.0979,0.1100,118.076,190320,4.0,2019,2019-11-22,[Robbie Williams],[Robbie Williams]
382852,2Fupsew1vcSKEBYMYrLXvi,Not Christmas - Bonus Track,The Christmas Present (Deluxe),1JDeu83sZU5nmRu2OTSk4F,['Robbie Williams'],['2HcwFjNelS49kFbfvMxQYw'],13,2,False,0.483,...,0.000000,0.0837,0.5840,167.999,205480,4.0,2019,2019-11-22,[Robbie Williams],[Robbie Williams]
382853,0cezJ0ZDfOTCCrVhVgcl3Z,Merry Kissmas - Bonus Track,The Christmas Present (Deluxe),1JDeu83sZU5nmRu2OTSk4F,['Robbie Williams'],['2HcwFjNelS49kFbfvMxQYw'],14,2,False,0.801,...,0.000000,0.0921,0.8810,123.007,162227,4.0,2019,2019-11-22,[Robbie Williams],[Robbie Williams]


In [15]:
date = "at 2023-03-27T21:30:00+0100"

regex_pattern = r"at\s(.*?)" 

re.match(regex_pattern, date).group(1)

'at'

In [6]:
import re

get_name_regex =  r"(.*?)\s(at\s\d{4}-\d{2}-\d{2})"

string = "Something at Somewhere at 2023-10-10 T/19674783hdkwhbf8479"

re.match(get_name_regex, string).group(1)

'Something at Somewhere'

In [10]:
import requests

params = {"min_date": '2023-03-01',
          "max_date": '2023-03-31',
          "city" : 'Lisboa',
          "artist_name" : 'amy winehouse',
          "song_name": 'Rehab',
          "key": 'BQDKMU18vd_mc6lpS29b0n5m807O-nURUe_MGotAU9GnU3WmUXxAHk9k98N1HDaHFUGcct2CHDjcXUI_8Q_-CA-aKl4fMs0SDAiyZ4Z8uXIq8EVT2lj0t_Yxs8E1gm-SKtuyE7j4X62Omk2NO7qJP3bAdDsjK8bkfjJYypeGl4p4eBTpog'}


twc_api = 'https://terrible-with-concerts-hzefje2pbq-ew.a.run.app/predict'
response = requests.get(twc_api, params=params).json()
response

{'results': [{'best_matched_artist': 'Marinho',
   'suggested_gig_name': 'Marinho at Cossoul',
   'suggested_gig_date': '2023-03-25',
   'ticket_url': None,
   'latitude': None,
   'longitude': None},
  {'best_matched_artist': 'Baleia Baleia Baleia',
   'suggested_gig_name': 'Baleia Baleia Baleia and Fugly',
   'suggested_gig_date': '2023-03-17',
   'ticket_url': None,
   'latitude': None,
   'longitude': None},
  {'best_matched_artist': 'Simone Locarni',
   'suggested_gig_name': 'Simone Locarni at AveNew',
   'suggested_gig_date': '2023-03-22',
   'ticket_url': None,
   'latitude': None,
   'longitude': None},
  {'best_matched_artist': 'Нервы',
   'suggested_gig_name': 'Нервы',
   'suggested_gig_date': '2023-03-23',
   'ticket_url': None,
   'latitude': None,
   'longitude': None},
  {'best_matched_artist': 'The Comet Is Coming',
   'suggested_gig_name': 'The Comet Is Coming',
   'suggested_gig_date': '2023-03-24',
   'ticket_url': 'https://www.bandsintown.com/e/103801964?app_id=cfc7f

In [17]:
import requests

params = {"min_date": '2023-03-01',
          "max_date": '2023-03-31',
          "city" : 'Lisboa',
          "artist_name" : 'radiohead',
          "song_name": 'climbing up the walls',
          "key": 'BQA9KrF_bWPlgYI_cCzl_nhXVA2MiplOb1EQ9o3fbqd8VjwsJLB1PH9gjUT2YsDZ61xMURifZMaxa5YJJT9FGOzWxBLu_hOb2Jjg9YvC_nD9SHidoqeoDoJPtVK6-jA_jidrVqjfvjVueB2iC-9BIlC72pgXBc7tEzzyEJyzhDiQeZe6Ng'}


twc_api = 'https://terrible-with-concerts-hzefje2pbq-ew.a.run.app/predict'
response = requests.get(twc_api, params=params).json()
response['results']

[{'best_matched_artist': 'Bad Bad Mary',
  'suggested_gig_name': 'Bad Bad Mary at Music Box',
  'suggested_gig_date': '2023-03-17',
  'ticket_url': None,
  'latitude': None,
  'longitude': None},
 {'best_matched_artist': 'Avatar',
  'suggested_gig_name': 'Avatar',
  'suggested_gig_date': '2023-03-26',
  'ticket_url': 'https://www.bandsintown.com/e/102711791?app_id=cfc7fb20ddfa769f9def3f4dc02bc45d&came_from=267&utm_medium=api&utm_source=public_api&utm_campaign=event',
  'latitude': '-9.110635499999999',
  'longitude': '38.7571856'},
 {'best_matched_artist': 'Marinho',
  'suggested_gig_name': 'Marinho at Cossoul',
  'suggested_gig_date': '2023-03-25',
  'ticket_url': None,
  'latitude': None,
  'longitude': None},
 {'best_matched_artist': 'André Sardet',
  'suggested_gig_name': 'André Sardet at Teatro Tivoli BBVA',
  'suggested_gig_date': '2023-03-18',
  'ticket_url': None,
  'latitude': None,
  'longitude': None},
 {'best_matched_artist': 'Wolf Manhattan',
  'suggested_gig_name': 'Wolf 

In [18]:
lista = response['results']

In [21]:
import pandas as pd

lista_df = pd.DataFrame(lista)

In [34]:
lat = None
while lat is None:
    for i, row in lista_df.iterrows():
        if row['latitude'] is not None:
            lat = row['latitude']
            break

In [35]:
lat 

'-9.110635499999999'